In [6]:
import pandas as pd 
import numpy as np

In [7]:
raw_sessiondata = pd.read_csv("20210113-133853.csv", skiprows=6, delimiter = ';')

In [8]:
#Add a column for 'timestamps'
import time
import datetime
import ciso8601

timestamp = []
for i in range(len(raw_sessiondata.index)):
    t = raw_sessiondata['PC-TIME'].iloc[i]
    ts= ciso8601.parse_datetime(t)
    if i == 0:
        t0= time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(t0-t0,3))
    else:
        tnow = time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(tnow-t0,3))

raw_sessiondata['timestamps'] = timestamp

In [9]:
df_group_info = raw_sessiondata.groupby('+INFO')
df_group_type = raw_sessiondata.groupby('TYPE')

## Audio Timestamp

In [10]:
# Find Audio timestamps ('BNC1High'), process so no duplicates
df_bnc1high = df_group_info.get_group('BNC1High')
a = df_bnc1high['timestamps'].diff().fillna(df_bnc1high['timestamps']) > 10
timestamps_bnc1high = df_bnc1high.loc[a].timestamps.to_numpy()

In [21]:
timestamps_10ms = timestamps_bnc1high*100

In [23]:
rounded_timestamps_10ms = [int(round(timestamps_10ms[i])) for i in range(len(timestamps_10ms))]
print(rounded_timestamps_10ms)

[2137, 6215, 8293, 11370, 13449, 16547, 19628, 22718, 24806, 27886, 31966, 36040, 39109, 41184, 45255, 48327, 51397, 53468, 56547, 58618, 61693, 65765, 68839, 72925, 75008, 77079, 81152, 85229, 87302, 91373, 95447, 99517, 101590, 105665, 107739, 111807, 113884, 115955, 120025, 122100, 125174, 128249, 130324, 132397, 135468, 139544, 142617, 144690, 148765, 151838, 155913, 158991, 161066, 163142, 167216, 171286, 174359, 176435, 178513, 181588, 184667, 188743, 192817, 195891, 197965, 201038, 205110, 209184, 211275, 214356, 217440, 219517, 222592, 226663, 230751]


In [31]:
sampled_audio_10ms = np.zeros(rounded_timestamps_10ms[-1]+1)
np.put(sampled_audio_10ms,rounded_timestamps_10ms,1)

#### Checking Accuracy; should have the result as 'rounded_timestamps_10ms'

In [34]:
np.where(sampled_audio_10ms == 1)

(array([  2137,   6215,   8293,  11370,  13449,  16547,  19628,  22718,
         24806,  27886,  31966,  36040,  39109,  41184,  45255,  48327,
         51397,  53468,  56547,  58618,  61693,  65765,  68839,  72925,
         75008,  77079,  81152,  85229,  87302,  91373,  95447,  99517,
        101590, 105665, 107739, 111807, 113884, 115955, 120025, 122100,
        125174, 128249, 130324, 132397, 135468, 139544, 142617, 144690,
        148765, 151838, 155913, 158991, 161066, 163142, 167216, 171286,
        174359, 176435, 178513, 181588, 184667, 188743, 192817, 195891,
        197965, 201038, 205110, 209184, 211275, 214356, 217440, 219517,
        222592, 226663, 230751]),)

Correct yeah!

## Creating DataFrame: 'Sampled_10ms.audio'

In [35]:
# intialise data of lists. 
data = {'Sampled_10ms.audio':sampled_audio_10ms} 
# Create DataFrame 
df_audio = pd.DataFrame(data)

In [36]:
df_audio

,Sampled_10ms.audio
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
230747,0.0
230748,0.0
230749,0.0
230750,0.0


In [37]:
df_audio.to_hdf('Sampled_10ms.h5', key = 'dfaudio')